<a href="https://colab.research.google.com/github/khs5696/AI504/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Hyperparameter

In [2]:
batch_size = 128
learning_rate = 0.0002
epochs = 20

Dataset

In [3]:
train = datasets.MNIST('./', train=True, transform=transforms.ToTensor(), download=True)
test = datasets.MNIST('./', train=False, transform=transforms.ToTensor(), download=True)
train, valid = torch.utils.data.random_split(train, [50000, 10000])

train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 45305337.21it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 117379548.28it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23286456.86it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20288103.05it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
transforms_train = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transforms_test = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar_train = datasets.CIFAR10(root='./', train=True, download=True, transform=transforms_train)
cifar_test = datasets.CIFAR10(root='./', train=False, download=True, transform=transforms_test)
cifar_loader = {}
cifar_loader['train'] = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=4)
cifar_loader['test'] = DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=4)

100%|██████████| 170498071/170498071 [00:05<00:00, 29134076.96it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Model

In [5]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=0),
      nn.BatchNorm2d(32),
      nn.ReLU(),
    )

    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, stride=1, padding=0),
      nn.BatchNorm2d(64),
      nn.ReLU(),
    )

    self.fc = nn.Linear(64 * 16 * 16, 10)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = torch.flatten(x, 1)
    output = self.fc(x)
    return output

Train

In [6]:
model = Model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [8]:
model.train()

for epoch in range(epochs):
  epoch_loss = 0
  print("-"*10)
  print("Epochs {}/{}".format(epoch+1, epochs))

  for data, label in train_dataloader:
    data, label = data.to(device), label.to(device)
    output = model(data)
    loss = criterion(output, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  epoch_loss += loss
  print("Train_loss : {}".format(epoch_loss))

----------
Epochs 1/20
Train_loss : 0.18714705109596252
----------
Epochs 2/20
Train_loss : 0.21925446391105652
----------
Epochs 3/20
Train_loss : 0.32460319995880127
----------
Epochs 4/20
Train_loss : 0.20670290291309357
----------
Epochs 5/20
Train_loss : 0.17359289526939392
----------
Epochs 6/20
Train_loss : 0.14020200073719025
----------
Epochs 7/20
Train_loss : 0.1866656243801117
----------
Epochs 8/20
Train_loss : 0.27561062574386597
----------
Epochs 9/20
Train_loss : 0.22048798203468323
----------
Epochs 10/20
Train_loss : 0.18693402409553528
----------
Epochs 11/20
Train_loss : 0.1536802500486374
----------
Epochs 12/20
Train_loss : 0.1600993573665619
----------
Epochs 13/20
Train_loss : 0.19075573980808258
----------
Epochs 14/20
Train_loss : 0.23639197647571564
----------
Epochs 15/20
Train_loss : 0.08616496622562408
----------
Epochs 16/20
Train_loss : 0.257250040769577
----------
Epochs 17/20
Train_loss : 0.05961541086435318
----------
Epochs 18/20
Train_loss : 0.126809

In [12]:
model.eval()
accuracy = 0

for data, label in test_dataloader:
  data, label = data.to(device), label.to(device)
  output = torch.argmax(model(data), dim=1)
  accuracy += (output == label).sum().item()

print(accuracy / len(test_dataloader.dataset) * 100)

97.13000000000001


ResNet

In [17]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

    def down_sampling(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.out_channels - self.in_channels))
        out = nn.MaxPool2d(2, stride=self.stride)(out)
        return out

    def forward(self, x):
        shortcut = x # this will be used to build residual connection.

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.stride != 1:
            shortcut = self.down_sampling(x)

        out += shortcut # residual connection
        out = self.relu(out)
        return out

In [18]:
class ResNet(nn.Module):
    def __init__(self, num_layers, block, num_classes=10):
        super(ResNet, self).__init__()
        self.num_layers = num_layers

        #input(channel:3) -> (conv 3x3) -> (bn) -> (relu) -> output(channel:16)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # feature map size = 16x32x32
        self.layers_2n = self.get_layers(block, 16, 16, stride=1)
        # feature map size = 32x16x16
        self.layers_4n = self.get_layers(block, 16, 32, stride=2)
        # feature map size = 64x8x8
        self.layers_6n = self.get_layers(block, 32, 64, stride=2)

        # output layers
        self.avg_pool = nn.AvgPool2d(8, stride=1)
        self.fc_out = nn.Linear(64, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def get_layers(self, block, in_channels, out_channels, stride):
        if stride == 2:
            down_sample = True
        else:
            down_sample = False

        layer_list = nn.ModuleList([block(in_channels, out_channels, stride, down_sample)])

        for _ in range(self.num_layers - 1):
            layer_list.append(block(out_channels, out_channels))

        return nn.Sequential(*layer_list)


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layers_2n(x)
        x = self.layers_4n(x)
        x = self.layers_6n(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_out(x)

        return x

In [19]:
def resnet18():
    block = ResidualBlock
    model = ResNet(3, block)
    return model
def resnet32():
    block = ResidualBlock
    model = ResNet(5, block)
    return model

In [20]:
net = resnet18().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
decay_epoch = [32000, 48000]
step_lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)

In [21]:
import time
start_time = time.time()

net.train()

step = 0
epochs = 0
losses = []

while step < 64000:

    train_loss = 0.0
    correct = 0.0
    total = 0.0

    for batch_idx, (x, y) in enumerate(cifar_loader['train']):
        step += 1
        step_lr_scheduler.step()
        x, y = x.to(device), y.to(device)
        out = net(x)
        loss = criterion(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (out.argmax(1) == y).float().sum().item()
        total += x.size(0)
        train_loss += loss.item()

    losses.append(train_loss)
    epochs += 1

    print("Epoch[{:d} ({:d}/64000) ({:.4f}sec)] loss: {:.2f} acc: {:.2f}".format(epochs, step, time.time()-start_time, train_loss, 100.*correct/total))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before

Epoch[1 (391/64000) (22.6064sec)] loss: 659.64 acc: 38.45
Epoch[2 (782/64000) (45.1633sec)] loss: 426.82 acc: 61.09
Epoch[3 (1173/64000) (67.8785sec)] loss: 336.28 acc: 69.76
Epoch[4 (1564/64000) (89.7628sec)] loss: 293.58 acc: 73.67
Epoch[5 (1955/64000) (112.2177sec)] loss: 265.68 acc: 76.26
Epoch[6 (2346/64000) (135.1166sec)] loss: 244.22 acc: 78.14
Epoch[7 (2737/64000) (157.5045sec)] loss: 227.42 acc: 79.90
Epoch[8 (3128/64000) (180.1812sec)] loss: 214.10 acc: 81.04
Epoch[9 (3519/64000) (202.7403sec)] loss: 205.63 acc: 81.67
Epoch[10 (3910/64000) (224.2537sec)] loss: 193.49 acc: 83.09
Epoch[11 (4301/64000) (247.0863sec)] loss: 187.82 acc: 83.21
Epoch[12 (4692/64000) (269.7395sec)] loss: 180.14 acc: 84.14
Epoch[13 (5083/64000) (292.2314sec)] loss: 174.57 acc: 84.65
Epoch[14 (5474/64000) (314.2126sec)] loss: 168.02 acc: 85.06
Epoch[15 (5865/64000) (337.0401sec)] loss: 162.11 acc: 85.50
Epoch[16 (6256/64000) (359.8452sec)] loss: 158.38 acc: 85.84
Epoch[17 (6647/64000) (381.5541sec)] lo

KeyboardInterrupt: ignored